<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Deteccion_Anomalias_Auscultacion_Presas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets

# Diccionarios para almacenar por tipo de archivo
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Función para detectar tipo de instrumento por nombre
def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return None  # Puntos fijos sin margen, no válido
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# --- Widget de carga de archivos ---
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)

output = widgets.Output()

# Función principal de carga
def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        for nombre_archivo, archivo_info in archivos.items():
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                if not instrumento:
                    print(f"❌ Instrumento no reconocido o mal nombrado: {nombre_archivo}")
                    continue

                # Cargar el archivo
                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                    datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (CSV)")
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                    datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (XLSX)")
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")
            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

        mostrar_menu()

# Función de visualización dinámica
def mostrar_menu():
    opciones = []
    for origen in ['csv', 'xlsx']:
        for instrumento in instrumentos:
            opciones.append(f"{instrumento} ({origen})")

    selector = widgets.Dropdown(
        options=opciones,
        description='Seleccionar DataFrame:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )

    def mostrar_datos(change):
        clear_output(wait=True)
        display(upload_widget, output)
        seleccion = selector.value
        instrumento, origen = seleccion.split(" ")
        instrumento = instrumento.strip()
        origen = origen.strip("()")

        print(f"📊 Mostrando: {instrumento.upper()} ({origen.upper()})")
        if origen == "csv":
            display(datos_csv[instrumento].head())
        else:
            display(datos_xlsx[instrumento].head())

    selector.observe(mostrar_datos, names='value')
    display(selector)

# Conectar evento
upload_widget.observe(cargar_archivos, names='value')

# Mostrar interfaz
display(upload_widget)
display(output)

FileUpload(value={}, accept='.csv,.xlsx', description='Subir archivos', multiple=True, style=ButtonStyle(butto…

Output()

In [ ]:
# SCRIPT 5: Detección de Anomalías en Series Temporales de Instrumentos de Auscultación
# Modular, independiente, requiere datos_csv y datos_xlsx ya cargados

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from sklearn.ensemble import IsolationForest

# --- Selectores reutilizados (idénticos a scripts previos) ---
instrumento_dropdown = widgets.Dropdown(
    options=[
        "Puntos Fijos", "Piezómetros Eléctricos", "Piezómetros Casagrande", "Inclinómetros",
        "Celdas de Asentamiento", "Freatímetros", "Extensómetros"
    ],
    value="Puntos Fijos", description="Instrumento:"
)
origen_dropdown = widgets.Dropdown(options=["CSV", "XLSX"], value="CSV", description="Origen:")

# Selectores específicos (estructura igual a scripts anteriores)
margen_dropdown = widgets.Dropdown(description="Margen:", options=[])
punto_dropdown = widgets.Dropdown(description="Punto Fijo:", options=[])
variable_pf_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_pf_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])

progresiva_dropdown = widgets.Dropdown(description="Progresiva:", options=[])
piezometro_dropdown = widgets.Dropdown(description="Piezómetro:", options=[])
variable_pe_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_pe_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])

margen_cg_dropdown = widgets.Dropdown(description="Margen:", options=[])
pz_cg_dropdown = widgets.Dropdown(description="Piezómetro:", options=[])
variable_cg_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_cg_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])

inclinometro_dropdown = widgets.Dropdown(description="Inclinómetro:", options=[])
anio_inc_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
eje_dropdown = widgets.Dropdown(options=["A+", "A-", "B+", "B-"], value="A+", description="Eje:")

progresiva_ca_dropdown = widgets.Dropdown(description="Progresiva:", options=[])
celda_dropdown = widgets.Dropdown(description="Celda:", options=[])
variable_ca_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_ca_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])

freatimetro_dropdown = widgets.Dropdown(description="Freatímetro:", options=[])
variable_fr_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_fr_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])

extensometro_dropdown = widgets.Dropdown(description="Extensómetro:", options=[])
variable_ex_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_ex_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])

# --- Parámetros de detección y controles ---
metodo_dropdown = widgets.Dropdown(
    options=[("IQR (estadístico robusto)", "IQR"), ("Isolation Forest (ML)", "IForest")],
    value="IQR", description="Método:"
)
iqr_factor_slider = widgets.FloatSlider(value=1.5, min=1, max=5, step=0.1, description='Factor IQR:')
iforest_cont_slider = widgets.FloatSlider(value=0.05, min=0.01, max=0.3, step=0.01, description='Contamination:')
boton_detectar = widgets.Button(description="Detectar Anomalías", button_style="danger")
output = widgets.Output()
descargar_csv_btn = widgets.Button(description="Descargar anomalías (CSV)", button_style="info")
output_csv = widgets.Output()

# --- Helper: Agregar "Cota Nivel Freático" ---
def _agregar_cota_nf(variables):
    if "Cota Nivel Freático" not in variables:
        variables.append("Cota Nivel Freático")
    return variables

# === Funciones de actualización de selectores (idénticas a Script 4, solo muestro PE/CG/Freatímetros aquí) ===
def actualizar_opciones_pe(change=None):
    df = (datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]).copy()
    if df.empty or 'PROGRESIVA' not in df.columns:
        progresiva_dropdown.options = []
        piezometro_dropdown.options = []
        variable_pe_dropdown.options = []
        anio_pe_dropdown.options = []
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique())
    if progresiva_dropdown.options:
        progresiva_dropdown.value = progresiva_dropdown.options[0]
    actualizar_piezometros_pe()
    columnas_excluir = ['FECHA', 'PROGRESIVA', 'PIEZOMETRO']
    variables = [c for c in df.select_dtypes(include='number').columns if c not in columnas_excluir]
    variables = _agregar_cota_nf(variables)
    variable_pe_dropdown.options = variables
    if variables:
        variable_pe_dropdown.value = variables[0]
    anio_pe_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_pe(change=None):
    df = (datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]).copy()
    if df.empty or 'PROGRESIVA' not in df.columns or 'PIEZOMETRO' not in df.columns:
        piezometro_dropdown.options = []
        return
    piezos = sorted(df[df['PROGRESIVA'] == progresiva_dropdown.value]['PIEZOMETRO'].dropna().unique())
    piezometro_dropdown.options = ["Todos"] + list(piezos)

def actualizar_opciones_cg(change=None):
    df = (datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]).copy()
    if df.empty or 'MARGEN' not in df.columns:
        margen_cg_dropdown.options = []
        pz_cg_dropdown.options = []
        variable_cg_dropdown.options = []
        anio_cg_dropdown.options = []
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    margen_cg_dropdown.options = sorted(df['MARGEN'].dropna().unique())
    if margen_cg_dropdown.options:
        margen_cg_dropdown.value = margen_cg_dropdown.options[0]
    actualizar_piezometros_cg()
    columnas_excluir = ['FECHA', 'MARGEN', 'PIEZOMETRO']
    variables = [c for c in df.select_dtypes(include='number').columns if c not in columnas_excluir]
    variables = _agregar_cota_nf(variables)
    variable_cg_dropdown.options = variables
    if variables:
        variable_cg_dropdown.value = variables[0]
    anio_cg_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_cg(change=None):
    df = (datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]).copy()
    if df.empty or 'MARGEN' not in df.columns or 'PIEZOMETRO' not in df.columns:
        pz_cg_dropdown.options = []
        return
    piezos = sorted(df[df['MARGEN'] == margen_cg_dropdown.value]['PIEZOMETRO'].dropna().unique())
    pz_cg_dropdown.options = ["Todos"] + list(piezos)

def actualizar_opciones_fr():
    df = (datos_csv["freatimetros"] if origen_dropdown.value == "CSV" else datos_xlsx["freatimetros"]).copy()
    if df.empty:
        freatimetro_dropdown.options = []
        variable_fr_dropdown.options = []
        anio_fr_dropdown.options = []
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    freatimetro_dropdown.options = ["Todos"] + sorted(df['FREATIMETRO'].dropna().unique())
    variables = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'FREATIMETRO']]
    variables = _agregar_cota_nf(variables)
    variable_fr_dropdown.options = variables
    anio_fr_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

# (Incluye las funciones completas de actualización de selectores de Script 4 aquí...)

def actualizar_controles_visibles(change=None):
    tipo = instrumento_dropdown.value
    for w in [margen_dropdown, punto_dropdown, variable_pf_dropdown, anio_pf_dropdown,
              progresiva_dropdown, piezometro_dropdown, variable_pe_dropdown, anio_pe_dropdown,
              margen_cg_dropdown, pz_cg_dropdown, variable_cg_dropdown, anio_cg_dropdown,
              inclinometro_dropdown, anio_inc_dropdown, eje_dropdown,
              progresiva_ca_dropdown, celda_dropdown, variable_ca_dropdown, anio_ca_dropdown,
              freatimetro_dropdown, variable_fr_dropdown, anio_fr_dropdown,
              extensometro_dropdown, variable_ex_dropdown, anio_ex_dropdown]:
        w.layout.display = 'none'
    if tipo == "Puntos Fijos":
        margen_dropdown.layout.display = 'flex'
        punto_dropdown.layout.display = 'flex'
        variable_pf_dropdown.layout.display = 'flex'
        anio_pf_dropdown.layout.display = 'flex'
        actualizar_opciones_pf()
    elif tipo == "Piezómetros Eléctricos":
        progresiva_dropdown.layout.display = 'flex'
        piezometro_dropdown.layout.display = 'flex'
        variable_pe_dropdown.layout.display = 'flex'
        anio_pe_dropdown.layout.display = 'flex'
        actualizar_opciones_pe()
    elif tipo == "Piezómetros Casagrande":
        margen_cg_dropdown.layout.display = 'flex'
        pz_cg_dropdown.layout.display = 'flex'
        variable_cg_dropdown.layout.display = 'flex'
        anio_cg_dropdown.layout.display = 'flex'
        actualizar_opciones_cg()
    elif tipo == "Inclinómetros":
        inclinometro_dropdown.layout.display = 'flex'
        anio_inc_dropdown.layout.display = 'flex'
        eje_dropdown.layout.display = 'flex'
        actualizar_opciones_inc()
    elif tipo == "Celdas de Asentamiento":
        progresiva_ca_dropdown.layout.display = 'flex'
        celda_dropdown.layout.display = 'flex'
        variable_ca_dropdown.layout.display = 'flex'
        anio_ca_dropdown.layout.display = 'flex'
        actualizar_opciones_ca()
    elif tipo == "Freatímetros":
        freatimetro_dropdown.layout.display = 'flex'
        variable_fr_dropdown.layout.display = 'flex'
        anio_fr_dropdown.layout.display = 'flex'
        actualizar_opciones_fr()
    elif tipo == "Extensómetros":
        extensometro_dropdown.layout.display = 'flex'
        variable_ex_dropdown.layout.display = 'flex'
        anio_ex_dropdown.layout.display = 'flex'
        actualizar_opciones_ex()

def mostrar_parametros_metodo(change=None):
    if metodo_dropdown.value == "IQR":
        iqr_factor_slider.layout.display = 'flex'
        iforest_cont_slider.layout.display = 'none'
    else:
        iqr_factor_slider.layout.display = 'none'
        iforest_cont_slider.layout.display = 'flex'
metodo_dropdown.observe(mostrar_parametros_metodo, names='value')

# =================== FUNCIÓN PRINCIPAL DE DETECCIÓN ===================
anomalies_df = None  # global para exportación

def detectar_anomalias(b=None):
    global anomalies_df
    with output:
        clear_output(wait=True)
        tipo = instrumento_dropdown.value
        origen = origen_dropdown.value
        fecha_col = "FECHA"
        variable = None
        df = None
        titulo = ""
        # --- Filtros por instrumento (idéntico a scripts previos, adaptado para "Cota Nivel Freático") ---
        if tipo == "Puntos Fijos":
            df = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
            if margen_dropdown.value == "Margen Derecha (MD)":
                df = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
            variable = variable_pf_dropdown.value
            if anio_pf_dropdown.value != "Todos":
                df = df.copy()
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_pf_dropdown.value)]
            if punto_dropdown.value != "Todos":
                df = df.copy()
                df = df[df['INSTRUMENTO'] == punto_dropdown.value]
            titulo = f"{margen_dropdown.value}: {variable}"
        elif tipo == "Piezómetros Eléctricos":
            df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
            variable = variable_pe_dropdown.value
            if progresiva_dropdown.value != "Todos":
                df = df.copy()
                df = df[df['PROGRESIVA'] == progresiva_dropdown.value]
            if piezometro_dropdown.value != "Todos":
                df = df.copy()
                df = df[df['PIEZOMETRO'] == piezometro_dropdown.value]
            if anio_pe_dropdown.value != "Todos":
                df = df.copy()
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_pe_dropdown.value)]
            titulo = f"{progresiva_dropdown.value} – {variable}"
        elif tipo == "Piezómetros Casagrande":
            df = datos_csv["piezometros_casagrande"] if origen == "CSV" else datos_xlsx["piezometros_casagrande"]
            variable = variable_cg_dropdown.value
            if margen_cg_dropdown.value != "Todos":
                df = df.copy()
                df = df[df['MARGEN'] == margen_cg_dropdown.value]
            if pz_cg_dropdown.value != "Todos":
                df = df.copy()
                df = df[df['PIEZOMETRO'] == pz_cg_dropdown.value]
            if anio_cg_dropdown.value != "Todos":
                df = df.copy()
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_cg_dropdown.value)]
            titulo = f"{margen_cg_dropdown.value} – {variable}"
        elif tipo == "Inclinómetros":
            df = datos_csv["inclinometros"] if origen == "CSV" else datos_xlsx["inclinometros"]
            fecha_col = 'Fecha'
            variable = eje_dropdown.value
            if inclinometro_dropdown.value != "Todos":
                df = df.copy()
                df = df[df['Inclinometro'] == inclinometro_dropdown.value]
            if anio_inc_dropdown.value != "Todos":
                df = df.copy()
                df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
                df = df[df['Fecha'].dt.year == int(anio_inc_dropdown.value)]
            titulo = f"{inclinometro_dropdown.value} – {variable}"
        elif tipo == "Celdas de Asentamiento":
            df = datos_csv["asentamiento"] if origen == "CSV" else datos_xlsx["asentamiento"]
            variable = variable_ca_dropdown.value
            if progresiva_ca_dropdown.value != "Todos":
                df = df.copy()
                df = df[df['PROGRESIVA'] == progresiva_ca_dropdown.value]
            if celda_dropdown.value != "Todas":
                df = df.copy()
                df = df[df['CELDA_DE_ASENTAMIENTO'] == celda_dropdown.value]
            if anio_ca_dropdown.value != "Todos":
                df = df.copy()
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_ca_dropdown.value)]
            titulo = f"{progresiva_ca_dropdown.value} – {variable}"
        elif tipo == "Freatímetros":
            df = datos_csv["freatimetros"] if origen == "CSV" else datos_xlsx["freatimetros"]
            variable = variable_fr_dropdown.value
            if freatimetro_dropdown.value != "Todos":
                df = df.copy()
                df = df[df['FREATIMETRO'] == freatimetro_dropdown.value]
            if anio_fr_dropdown.value != "Todos":
                df = df.copy()
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_fr_dropdown.value)]
            titulo = f"{freatimetro_dropdown.value} – {variable}"
        elif tipo == "Extensómetros":
            df = datos_csv["extensometro"] if origen == "CSV" else datos_xlsx["extensometro"]
            variable = variable_ex_dropdown.value
            if extensometro_dropdown.value != "Todos":
                df = df.copy()
                df = df[df['EXTENSOMETRO'] == extensometro_dropdown.value]
            if anio_ex_dropdown.value != "Todos":
                df = df.copy()
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_ex_dropdown.value)]
            titulo = f"{extensometro_dropdown.value} – {variable}"
        else:
            print("⚠️ Instrumento no reconocido."); return

        if df is None or variable is None or fecha_col not in df.columns:
            print("⚠️ Faltan datos o columnas requeridas."); return

        df = df.copy()
        df[fecha_col] = pd.to_datetime(df[fecha_col], dayfirst=True, errors='coerce')
        df = df.sort_values(fecha_col)

        # Lógica para "Cota Nivel Freático"
        if variable == "Cota Nivel Freático":
            if "Cota Nivel Freático" in df.columns:
                serie_col = "Cota Nivel Freático"
            elif "COTA_NF" in df.columns:
                serie_col = "COTA_NF"
            else:
                print("No existe la columna 'Cota Nivel Freático' ni 'COTA_NF' en los datos cargados para este instrumento.")
                anomalies_df = None
                return
        else:
            serie_col = variable

        df[serie_col] = pd.to_numeric(df[serie_col], errors='coerce')
        df = df.dropna(subset=[fecha_col, serie_col])
        if df.empty:
            print("⚠️ No hay datos válidos para analizar."); anomalies_df = None; return

        # Agrupar por fecha para evitar duplicados
        df_group = df.groupby(fecha_col, as_index=False)[serie_col].mean()
        x = df_group[fecha_col]
        y = df_group[serie_col]

        # --- Detección de anomalías según método ---
        metodo = metodo_dropdown.value
        if metodo == "IQR":
            q1 = y.quantile(0.25)
            q3 = y.quantile(0.75)
            iqr = q3 - q1
            factor = iqr_factor_slider.value
            lower = q1 - factor * iqr
            upper = q3 + factor * iqr
            mask_anom = (y < lower) | (y > upper)
            metodo_label = f"IQR (factor={factor})"
        else:  # Isolation Forest
            model = IsolationForest(contamination=iforest_cont_slider.value, random_state=42)
            X = y.values.reshape(-1, 1)
            y_pred = model.fit_predict(X)
            mask_anom = (y_pred == -1)
            metodo_label = f"Isolation Forest (contamination={iforest_cont_slider.value})"

        # --- Visualización ---
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x, y=y, mode='lines+markers', name='Serie original'))
        if mask_anom.sum() > 0:
            fig.add_trace(go.Scatter(
                x=x[mask_anom], y=y[mask_anom], mode='markers', name='Anomalías',
                marker=dict(color='red', size=12, symbol="x")
            ))
        else:
            print("No se detectaron anomalías con el método y parámetros seleccionados.")
        fig.update_layout(
            title=f"Detección de Anomalías – {titulo}<br><sub>Método: {metodo_label}</sub>",
            xaxis_title="Fecha", yaxis_title=serie_col,
            legend_title="Serie", width=950, height=500
        )
        fig.show()

        # --- Tabla de anomalías para exportación ---
        anomalies_df = df_group.loc[mask_anom, [fecha_col, serie_col]].copy()
        if not anomalies_df.empty:
            print(f"Anomalías detectadas: {len(anomalies_df)}")
            display(anomalies_df)
        else:
            print("No hay registros anómalos para exportar.")

def descargar_csv_anomalias(b=None):
    from google.colab import files
    with output_csv:
        clear_output(wait=True)
        global anomalies_df
        if anomalies_df is None or anomalies_df.empty:
            print("No hay anomalías para exportar.")
            return
        anomalies_df.to_csv("anomalias_detectadas.csv", index=False)
        files.download("anomalias_detectadas.csv")
        print("Descarga iniciada.")

# --- Conectar eventos y mostrar interfaz ---
instrumento_dropdown.observe(actualizar_controles_visibles, names='value')
origen_dropdown.observe(actualizar_controles_visibles, names='value')
margen_dropdown.observe(actualizar_opciones_pf, names='value')
progresiva_dropdown.observe(actualizar_piezometros_pe, names='value')
margen_cg_dropdown.observe(actualizar_piezometros_cg, names='value')
progresiva_ca_dropdown.observe(actualizar_celdas_ca, names='value')
boton_detectar.on_click(detectar_anomalias)
descargar_csv_btn.on_click(descargar_csv_anomalias)

# --- Display interfaz ---
display(HTML("<h3 style='color:#d32f2f'>Detección de Anomalías en Series Temporales</h3>"))
display(widgets.HBox([instrumento_dropdown, origen_dropdown]))
display(widgets.HBox([
    margen_dropdown, punto_dropdown,
    progresiva_dropdown, piezometro_dropdown,
    margen_cg_dropdown, pz_cg_dropdown,
    inclinometro_dropdown,
    progresiva_ca_dropdown, celda_dropdown,
    freatimetro_dropdown,
    extensometro_dropdown
]))
display(widgets.HBox([
    variable_pf_dropdown, variable_pe_dropdown, variable_cg_dropdown,
    anio_pf_dropdown, anio_pe_dropdown, anio_cg_dropdown,
    anio_inc_dropdown, eje_dropdown,
    variable_ca_dropdown, anio_ca_dropdown,
    variable_fr_dropdown, anio_fr_dropdown,
    variable_ex_dropdown, anio_ex_dropdown
]))
display(widgets.HBox([metodo_dropdown, iqr_factor_slider, iforest_cont_slider]))
mostrar_parametros_metodo()
display(boton_detectar)
display(output)
display(widgets.HBox([descargar_csv_btn]))
display(output_csv)

# Inicializar controles
actualizar_controles_visibles()

Button(button_style='danger', description='Detectar Anomalías', style=ButtonStyle())

Output()

Output()